In [1]:
from pymongo import MongoClient
import pandas as pd
client = MongoClient()
db = client.openfoodfacts
collection = db.products

In [22]:
proj = {
    'product_name_fr' : 1, 
    'ingredients' : 1,
    '_id' : 1
}

addFields = { 
    
    'ingredients' : "$ingredients.id",
    
}


match = {
    
    'ingredients' : { 
        "$elemMatch" : { 
            "text" : { '$regex' : "tomate*", '$options' : "i" },
        } 
    },
    
    "product_name_fr" : { '$regex' : "sauce*", '$options' : "i" }
    
}



c_f = collection.aggregate([
    { '$match' : match } ,
    { '$project' : proj },
    { '$addFields' : addFields },
])

ingredients = pd.DataFrame(list(c_f))

In [23]:
ingredients

,_id,ingredients,product_name_fr
0,0000069162176,"[fr:INGREDIENTES, fr:agua, fr:harina de tngo t...",Quenelles de brochet sauce Nantua
1,00005010,"[en:tomato-puree, fr:beef mince meat, en:carro...",Organic tomato sauce bolognese
2,00021036,"[en:tomato-sauce, fr:à Itoignon kJ et à la car...",Tomato & Gorgonzola pasta sauce
3,00089227,"[en:tomato-pulp, fr:saucisse-de-porc, en:crush...",Tomato & Sausage Pasta Sauce
4,0011110791474,"[en:water, en:red-bell-pepper, en:distilled-vi...",Peri peri chili culinary hot sauce
...,...,...,...
2702,93201834,"[en:sugar, en:water, en:vinegar, en:tomato pas...",BBQ Sauce
2703,9300681008403,"[en:sugar, en:water, en:crushed-tomato, en:vin...",Sweet chili sauce
2704,9556041601214,"[fr:POISSON DE TYPE SARDINES, en:tomato-sauce,...",Ayam Sardines In Tomato Sauce
2705,9556041611121,"[en:water, en:sugar, en:thickener, en:crushed-...",Sauce aigre-douce Ayam™


Index(['_id', 'ingredients', 'product_name_fr'], dtype='object')


In [24]:
ingredient_list_id = []
ingredient_list_product_name_fr = []
ingredient_list_ingredients = []
ingredient_list_rank = []

for k in range(len(ingredients)) :
    rank = 1
    if isinstance(ingredients['ingredients'][k], list) :
        for ing in ingredients['ingredients'][k] :
            ingredient_list_ingredients.append(ing)
            ingredient_list_product_name_fr.append(ingredients['product_name_fr'][k])
            ingredient_list_id.append(ingredients['_id'][k])
            ingredient_list_rank.append(rank)
            rank += 1

In [30]:
df_ingredients = pd.DataFrame(list(zip(ingredient_list_id, ingredient_list_ingredients, ingredient_list_product_name_fr, ingredient_list_rank)), columns =['_id', 'ingredients', 'product_name_fr', 'ingredient_rank'])

In [38]:
df_ingredients.head(5)

,_id,ingredients,product_name_fr,ingredient_rank
0,0000069162176,fr:INGREDIENTES,Quenelles de brochet sauce Nantua,1
1,0000069162176,fr:agua,Quenelles de brochet sauce Nantua,2
2,0000069162176,fr:harina de tngo tierno carne de Lucio 13 %' ...,Quenelles de brochet sauce Nantua,3
3,0000069162176,fr:Lactosa y proteina léctea,Quenelles de brochet sauce Nantua,4
4,0000069162176,en:sal-tree,Quenelles de brochet sauce Nantua,5


In [39]:
def label_ingredient (row):
    if row['ingredients'][:2] == 'en' :
        return 0
    return 1

In [40]:
df_ingredients['unknown_ingredient'] = df_ingredients.apply(lambda row: label_ingredient(row), axis=1)

In [41]:
df_ingredients

,_id,ingredients,product_name_fr,ingredient_rank,unknown_ingredient
0,0000069162176,fr:INGREDIENTES,Quenelles de brochet sauce Nantua,1,1
1,0000069162176,fr:agua,Quenelles de brochet sauce Nantua,2,1
2,0000069162176,fr:harina de tngo tierno carne de Lucio 13 %' ...,Quenelles de brochet sauce Nantua,3,1
3,0000069162176,fr:Lactosa y proteina léctea,Quenelles de brochet sauce Nantua,4,1
4,0000069162176,en:sal-tree,Quenelles de brochet sauce Nantua,5,0
...,...,...,...,...,...
63472,9556041611978,en:water,Mi Goreng Sauce Wok,34,0
63473,9556041611978,en:salt,Mi Goreng Sauce Wok,35,0
63474,9556041611978,en:e150a,Mi Goreng Sauce Wok,36,0
63475,9556041611978,en:e150a,Mi Goreng Sauce Wok,37,0


In [42]:
df_ingredients.to_csv('ingredients.csv')

In [44]:
df_ingredients.iloc[102, 0:-1]

_id                             0013000009638
ingredients        en:armidon de maïs modifié
product_name_fr                Sauce barbecue
ingredient_rank                             7
Name: 102, dtype: object